In [9]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv("cian_dataset.csv", sep = ';')
df[['flat', 'int m2', 'float m2', 'floor']] = df['title'].str.split(',', expand= True )
df2 = df.drop('title', axis = 1)
df2['clean float part'] = df2['float m2'].apply(float_met)
df3 = df2.drop(columns = 'float m2', axis = 1)
df3['clean float part'] = df3['clean float part'].fillna(0) #Заполню пустые значения 0
df3['int m2'] = df3['int m2'].apply(for_int_m2)
df3['price_m'] = df3['price_m'].replace('₽/м²', '', regex=True)
df3['price_m'] = df3['price_m'].map(lambda x: int(''.join(x.split())))
df3['main_price'] = df3['main_price'].map(lambda price: int(''.join((str(price).strip('₽')).split())))
df3['flat square'] = df3['int m2']+df3['clean float part'] #скаладываем целую и дробную часть площади
df3 = df3.drop(columns = ['int m2', 'clean float part'], axis = 1) #удаляем более не нужные столбцы
df3['flat square'] = np.where(df3['flat square'] == 0.00, round(df3['main_price'] / df3['price_m'], 2), round(df3['flat square'],2))
#Принцип работы np.where(условие, если условие в искомой ячейке выполнено то подставь что я прошу, если не выполнено потсавь что я хочу)
df3['number of rooms'] = df3['flat'].apply(num_of_rooms)
df3 = df3.drop(columns = 'flat', axis =1)
df3[['floor', 'total of floors']] = df3['floor'].str.split('/', expand = True)
df3['total of floors'] = df3['total of floors'].apply(total_fl_proccesing)
df3.to_csv('итоггггг.csv', sep =',', encoding = 'utf8')

In [3]:
def float_met (value):
    try:
        if value is not None:
            value = value.split()
            value = value[0]
            if len(value) == 1:
                value = int(value)/10
                return value
            elif len(value) == 2 :
                value = int(value)/100
                return value
            else:
                return 0
    except ValueError:
            return (0)
#FUNCTION FOR PROCESSING FLOAT M2 AS THEY CONTAIN NOT ONLY FLOAT PART BUT OTHER GARBAGE I HAVE TO GET RID OFF 

In [4]:
def for_int_m2 (value): #Function in oreder to handle Type Error 
    try:
        if value is not None:
            value = re.sub(r'\D', '', value) #Библиотека re c помощью ее методов можно изабавиться от всех символов кроме числовых
            value = value.strip()
            value = int(value)
            return value
        else:
            return 0
    except ValueError:
        return 0

In [5]:
def num_of_rooms (value):
    one = ['однушка', 'однокомнатная', 'одн']
    two = ['двушка', 'двухкомнатная', 'двукомнатная', 'две']
    three = ['трёшка', 'трёх', 'три', 'трешка', 'трех', 'трехкомнатная', 'трёхкомнатная' ]
    four = ['четырехкомнатная', 'четверка', 'четвёрка']
    digits =  [str(x) for x in range (10)]
    if any(digit in value[0 ]for digit in digits):
#     if value[0] == [str(x) for x in range (10)]:
        value = int(value[0])
        return value 
    elif any(word in value.lower() for word in one ):
        return 1
    elif any(word in value.lower() for word in two):
        return 2
    elif any(word in value.lower() for word in three):
        return 3
    elif any(word in value.lower() for word in four):
        return 4

In [6]:
def total_fl_proccesing (value):
    try:
        if value is not None: 
            value = value.strip('этаж')
            value  =  int(value)
            return value
    except TypeError:
        pass